In [21]:
import networkx as nx
import os
from tqdm.notebook import tqdm
import scipy.stats
import numpy as np
import shutil
import pickle

In [2]:
src = 'data_processed'
dest = 'data_pearsonr'
destination = shutil.copytree(src, dest) 
dest = 'data_ncc'
destination = shutil.copytree(src, dest) 

In [2]:
root = "/home/polyxene/MLNS-project-datascienthyris/data_processed"
Files_found = []
for path, subdirs, files in os.walk(root):
    for name in files:
        Files_found.append(os.path.join(path, name))

In [22]:
for file in tqdm(Files_found):
    label, G = nx.read_gpickle(file)
    diG_pearson = G.to_directed()
    diG_ncc = G.to_directed()
    for node in diG_pearson.nodes():
        out_edges = list(diG_pearson.out_edges(nbunch=node, data="pearsonr"))
        edges_to_keep = sorted(out_edges, key=lambda tup: np.mean(tup[2]))[-3:]
        diG_pearson.remove_edges_from(out_edges)
        diG_pearson.add_weighted_edges_from(edges_to_keep)
    to_pickle = (label, diG_pearson)
    with open(file.replace("data_processed", "data_pearsonr"), "wb") as f:
            pickle.dump(to_pickle, f)
    
    for node in diG_ncc.nodes():
        out_edges = list(diG_ncc.out_edges(nbunch=node, data="norm cross correlation"))
        edges_to_keep = sorted(out_edges, key=lambda tup: np.mean(tup[2]))[-3:]
        diG_ncc.remove_edges_from(out_edges)
        diG_ncc.add_weighted_edges_from(edges_to_keep)
    to_pickle = (label, diG_ncc)
    with open(file.replace("data_processed", "data_ncc"), "wb") as f:
            pickle.dump(to_pickle, f)


  0%|          | 0/3050 [00:00<?, ?it/s]

In [23]:
shutil.make_archive("data_ncc", 'zip', "data_ncc")

'/home/polyxene/MLNS-project-datascienthyris/data_ncc.zip'

In [24]:
shutil.make_archive("data_pearsonr", 'zip', "data_pearsonr")

'/home/polyxene/MLNS-project-datascienthyris/data_pearsonr.zip'